In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import torch  
import whisper 
import openai
import tqdm
import sys
from dotenv import load_dotenv
load_dotenv()
# https://colab.research.google.com/github/AndreDalwin/Whisper2Summarize/blob/main/Whisper2Summarize_Colab_Edition.ipynb#scrollTo=y3CCY-m4Wbo6

True

In [3]:

audio = "data/audio.mp3" #Make sure you upload the audio file (mp3,wav,m4a) into the session storage!
model = "base" #possible options are 'tiny', 'base', 'small', 'medium', and 'large'
apikey = os.getenv('OPENAI_KEY')

In [4]:
def transcribe(audio,model_type):
    class _CustomProgressBar(tqdm.tqdm):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self._current = self.n  
            
        def update(self, n):
            super().update(n)
            self._current += n
            
            print("Audio Transcribe Progress: " + str(self._current) + "/" + str(self.total))
            
    devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
    print('devices',devices)
    model = whisper.load_model(model_type, device = devices)
    transcribe_module = sys.modules['whisper.transcribe']
    transcribe_module.tqdm.tqdm = _CustomProgressBar

    print("Beginning Transcribing Process...")
    result = model.transcribe(audio, verbose=None, fp16=False)
    transcribed = result["text"]
    with open("Transcript.txt", "w",encoding='utf-8') as text_file:
        text_file.write(transcribed)
        print("Saved Transcript to Transcript.txt")
    return transcribed

def gpt_process(transcript):
    openai.api_key = apikey
    print("Processing Transcript with GPT...")
    n=1300
    split = transcript.split()
    snippet= [' '.join(split[i:i+n]) for i in range(0,len(split),n)]
    ## For managing token limit
    summary=""
    previous=""
    for i in range(0, len(snippet), 1):
        print("Summarizing Transcribed Snippet {} of {}".format(i+1,len(snippet)))
        gpt_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "\"" + snippet[i] + "\"\n Rewrite the transcript above into notes. Do not summarize and keep every information. For additional context here is the previous rewritten message: \n " + previous }],
            temperature = 0.6,
        )
        previous = gpt_response['choices'][0]['message']['content']
        summary += gpt_response['choices'][0]['message']['content']

    with open("Summary.txt", "w",encoding='utf-8') as text_file:
        text_file.write(summary)
        print("Summarizing Completed.")
        print("Saved Summary to Summary.txt")


In [5]:
text = transcribe(audio,model)
gpt_process(text)

/root/code_Bao/stock_price_4_fun/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


devices cpu
Beginning Transcribing Process...
Audio Transcribe Progress: 1427/1427
Saved Transcript to Transcript.txt
Processing Transcript with GPT...
Summarizing Transcribed Snippet 1 of 1


RateLimitError: You exceeded your current quota, please check your plan and billing details.